## Group C
## 3 models trained on dataset 1 tested on dataset 2
## Ensemble of 3 models also tested

#### This notebook reads files:
##### AML_datasets.RData
##### fold1.100.ttest.csv             
##### rand50.of100.best.genes.csv
##### model.rF.1.RData
##### model.svm.rad.1.RData
##### model.svm.lin.1.RData

In [1]:
install.packages(c("e1071","randomForest"))
library(randomForest)
library(e1071) #svm

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

randomForest 4.7-1.1

Type rfNews() to see new features/changes/bug fixes.



In [2]:
load ("AML_datasets.RData")
rm (data.1, data.3, info.1, info.3)
 # data.2 <- read.table('data.2.csv')
 # info.2 <- read.table('info.2.csv')
data2.sub <- data.2[,1:3000]
info2.sub <- info.2[1:3000,]
# data2.sub <- read.table('data.2.sub.csv')
# info2.sub <- read.table('info.2.sub.csv')
dim(data2.sub)
dim(info2.sub)

[1] 12708  3000

[1] 3000    8

#### Load dataset 2, subset of first 3000 patients
(my kernel kept dying so I wanted less data)

#### proportion AML DataSet2: 0.31

In [3]:
### proportion AML in dataset 2
aml.overall <- rep(0, length(info.2$Disease))
aml.overall[which (info.2$Disease == "AML")] <- 1
len <- length(aml.overall)
sm <- sum(aml.overall)
sm
len
sm / len
rm(data.2,info.2)
ls()

[1] 2588

[1] 8348

[1] 0.3100144

[1] "aml.overall" "data2.sub"   "info2.sub"   "len"         "sm"

#### proportion AML in first 3000 patients of DataSet 2

In [4]:
#create AML / nonAML label
#look at balance of AML / nonAML
aml.prognosis <- rep(0, length(info2.sub$Disease))
aml.prognosis[which (info2.sub$Disease == "AML")] <- 1
len <- length(aml.prognosis)
sm <- sum(aml.prognosis)
sm
len
sm / len

[1] 985

[1] 3000

[1] 0.3283333

#### Read Gene lists used to train models in dataset 1

In [5]:
#100 genes used by SVM radial and SVM linear
genes100 <- read.table('fold1.100.ttest.csv')
genes100 <- genes100$V1[-1]
length(genes100)
genes100[1:10]

[1] 100

[1] "CREG1"   "ERLIN1"  "CD79A"   "POU2AF1" "LEF1"    "PAX5"    "CFD"    
 [8] "FAM46A"  "ATP8B4"  "PLAGL1"

In [6]:
#50 genes used by Random Forest
genes50 <- read.table('rand50.of100.best.genes.csv')
genes50  <- genes50$x
length(genes50)
genes50[1:10]

[1] 50

[1] "WARS"       "PTPRF"      "AP2M1"      "PFN1"       "JTB"       
 [6] "SPAG7"      "HNRNPA1P35" "ZNF146"     "RPL11"      "ZPR1"

#### Model 1: Random Forest
#### Error rate 24%

In [7]:
rf <- readRDS('model.rF.1.RData')
pred.rf <- predict(rf,t(data2.sub[genes50,]))
# str(pred.rf)
#pred.rf[1:10]
t3 <- table(aml.prognosis,pred.rf)
t3
error_rate <- (sum(t3) - sum(diag(t3)))  /  sum(t3)
error_rate

             pred.rf
aml.prognosis    0    1
            0 1962   53
            1  669  316

[1] 0.2406667

#### Model 2: SVM Radial
#### Error rate 6.6%

In [8]:
m2 <- readRDS('model.svm.rad.1.RData')

In [9]:
pred.svm.2 <- predict(m2,t(data2.sub[genes100,]))
# str(pred.svm.2)
# pred.svm.2[1:10]

In [10]:
svm.out.vec <- rep(0, length(pred.svm.2))
length(svm.out.vec)
svm.out.vec[which(pred.svm.2 == "AML")] <- 1
sum(svm.out.vec) / length(svm.out.vec)

[1] 3000

[1] 0.2936667

In [11]:
t2 <- table(aml.prognosis,svm.out.vec)
t2
error_rate <- (sum(t2) - sum(diag(t2)))  /  sum(t2)
error_rate

             svm.out.vec
aml.prognosis    0    1
            0 1968   47
            1  151  834

[1] 0.066

#### Model 3: SVM Linear
#### Error rate 28%

In [12]:
svm1 <- readRDS('model.svm.lin.1.RData')
pred.svm4 <- predict(svm1,t(data2.sub[genes100,]))

In [13]:
svm4.out.vec <- rep(0, length(pred.svm4))
length(svm4.out.vec)
svm4.out.vec[which(pred.svm4 == "AML")] <- 1
#Proportion of AML predicted
sum(svm4.out.vec) / length(svm4.out.vec)

[1] 3000

[1] 0.6106667

In [14]:
t4 <- table(aml.prognosis,svm4.out.vec)
t4
error_rate <- (sum(t4) - sum(diag(t4)))  /  sum(t4)
error_rate

             svm4.out.vec
aml.prognosis    0    1
            0 1165  850
            1    3  982

[1] 0.2843333

#### Ensemble method, takes binary prediction vectors, takes majority count for each patient prediction

In [15]:
#Ensemble of 3, majority count for binary classification (0,1)
maj.count <- function(a,b,c){
  vec <- NULL
  for (i in 1:length(a)) {
    temp <- a[i] + b[i] + c[i]
    vote <- 0
    if(temp >= 2){
      vote <- 1
    }
    vec <- c(vec,vote)
  }
  return(vec)
}

#### Calling ensemble on Random Forest, SVM 1, and SVM 2 model predictions

In [16]:

 rf.results <- round(as.numeric(as.character(pred.rf)))
# results2 <- round(as.numeric(as.character(pred.svm.2)))
# results3 <- round(as.numeric(as.character(pred.svm4)))

ret.ensemble <- maj.count(rf.results, svm.out.vec, svm4.out.vec)

length(ret.ensemble)
sample.indexes = 55:65
rf.results[sample.indexes]
svm.out.vec[sample.indexes]
svm4.out.vec[sample.indexes]
ret.ensemble[sample.indexes]

[1] 3000

[1] 0 0 0 0 0 0 0 0 0 0 0

[1] 0 0 0 0 0 0 0 0 0 0 0

[1] 1 1 1 0 1 1 1 1 1 1 0

[1] 0 0 0 0 0 0 0 0 0 0 0

##### Above, first 3 rows are individual model predictions, last row is ensemble prediction
## Ensemble error rate 7%

In [17]:
t5 <- table(ret.ensemble, aml.prognosis)
error_rate <- (sum(t5) - sum(diag(t5)))  /  sum(t5)
error_rate

[1] 0.07166667